In [17]:
import utils
from urllib.parse import urlparse

import os, shutil
from datetime import datetime

import sqlalchemy
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.mysql import LONGTEXT

import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', -1)  


from tqdm import tqdm
tqdm.pandas()

In [2]:
hostname="localhost"
dbname="apptio"
uname="sreejith"
pwd="password"
table = "events"


engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

In [3]:
connection = engine.raw_connection()
cursor = connection.cursor()

output_table = "events_clean"
command = "DROP TABLE IF EXISTS {};".format(output_table)
cursor.execute(command)

0

### Understanding the Clicked events
---

In [4]:
# url, documentReferer are the useful fields 

In [5]:
def get_domain(long_url):
    
    referrer_details = urlparse(str(long_url))
    return referrer_details.netloc

In [6]:
sql = "select documentReferer, url from " + table 
for chunk in pd.read_sql(sql, engine, chunksize=10000):
    
    chunk["from"] = chunk["documentReferer"].apply(get_domain)
    chunk["to"] = chunk["url"].apply(get_domain)
    
    del chunk["documentReferer"]
    del chunk["url"]
        
    events_clean_type_map = {x:types.TEXT for x in chunk.columns.tolist()}
    chunk.to_sql(name=output_table, con=engine, dtype=events_clean_type_map, index=False, if_exists='append')
    
    chunk = None
    del chunk

----

In [8]:
table = "events_clean"
sql = "select * from " + table 
df = pd.read_sql(sql, engine)

In [9]:
df.head(20)

,from,to
0,,s.amazon-adsystem.com
1,www.imdb.com,www.imdb.com
2,,www.facebook.com
3,,s.amazon-adsystem.com
4,www.imdb.com,www.imdb.com
5,,www.google.com
6,,www.google.com
7,,www.google.com
8,,www.facebook.com
9,,tpc.googlesyndication.com


In [22]:
df.groupby("from").size().reset_index(name="count").sort_values(by='count', ascending=False)[:30]

,from,count
0,,1236120
26272,www.google.com,380009
25249,www.facebook.com,269685
21490,www.amazon.com,93385
24731,www.ebay.com,83527
31451,www.reddit.com,81046
35440,www.youtube.com,57912
35335,www.yahoo.com,35391
1580,apps.facebook.com,35306
665,accounts.google.com,27403
